# 简介

> 指导文章：[01. 初识 LLM API：环境配置与多轮对话演示](https://github.com/Hoper-J/LLM-Guide-and-Demos/blob/master/Guide/01.%20初识%20LLM%20API：环境配置与多轮对话演示.md)

在这个 Notebook 中，我们将展示如何使用 API 与大模型进行交互。这是对 API 的一个基础演示，不涉及构建 AI 应用。

在线链接：[Kaggle](https://www.kaggle.com/code/aidemos/01-llm-api) | [Colab](https://colab.research.google.com/drive/1jFC_WIec7KW08GBxHuEHikcXWbbv3gSh?usp=share_link)

# 环境变量配置

为了保护 API 密钥的安全，需要将其设置为环境变量。环境变量允许我们在不将敏感信息写入代码的情况下访问它们。

你可以通过两种方式之一来设置环境变量：
1. 在终端中设置环境变量。
2. 在 Python 脚本中设置环境变量。

这里直接在 Python 中进行设置。

In [1]:
# 此方法仅在当前 Python 程序或 Notebook 中有效，其他程序或 Notebook 不会共享此设置。
# 国内大模型开启了新的篇章，考虑到以后可能会进行多模型的 API 输出对比，不再固定命名为 `OPENAI_API_KEY`。
import os

# 阿里云（通义千问） API 密钥在 OPENAI_API_KEY 设置
os.environ['OPENAI_API_KEY'] = 'your-api-key'

# 智谱 API
os.environ['ZHIPUAI_API_KEY'] = 'your-api-key'

# DEEPSEEK API
os.environ['DEEPSEEK_API_KEY'] = 'your-api-key'

# 获取环境变量

使用 `os.getenv()` 函数来获取环境变量的值，这样我们可以在代码中安全地访问API密钥。

In [2]:
# 获取API密钥
api_key = os.getenv('OPENAI_API_KEY')
# api_key = os.getenv('ZHIPUAI_API_KEY')
# api_key = os.getenv('DEEPSEEK_API_KEY')

# 打印密钥以确认它被成功读取
print(api_key)

# 安装所需库

接下来，我们需要安装 openai 库，用于与阿里云的大模型 API 进行交互。

In [ ]:
!pip install openai
#!pip install 'httpx<0.28.0' # 降级 httpx 以解决关键字 'proxies' 被移除的问题，最新的 openai 库不会引发该问题，故默认注释

根据 API 平台执行后续对应的代码块。

# 单轮对话演示

在这一部分，我们将通过API调用构建一个简单的单轮对话，你可以输入一个问题，模型将会返回一个响应。

## 阿里 API

In [4]:
from openai import OpenAI
import os

# 初始化 OpenAI 客户端，使用阿里云 DashScope API
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'), # 如果你没有配置环境变量，使用 api_key="your-api-key" 替换
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", # 这里使用的是阿里云的大模型，如果需要使用其他平台，请参考对应的开发文档后对应修改
)

# 调用 API 获取模型回复
response = client.chat.completions.create(
    model="qwen-turbo",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}]
    )

# 打印模型回复内容
print(response.choices[0].message.content)

我是阿里云开发的一款超大规模语言模型，我叫通义千问。


## 智谱 API

In [5]:
from openai import OpenAI
import os

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key=os.getenv('ZHIPUAI_API_KEY'),
    base_url="https://open.bigmodel.cn/api/paas/v4",
)

# 调用 API 获取模型回复
response = client.chat.completions.create(
    model="glm-4-plus",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}]
    )

# 打印模型回复内容
print(response.choices[0].message.content)

我是一个人工智能助手，旨在帮助解答问题和提供信息。你有什么问题或需要帮助的地方吗？


## DeepSeek API


In [6]:
from openai import OpenAI
import os

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key=os.getenv('DEEPSEEK_API_KEY'),  # 1
    base_url="https://api.deepseek.com",  # 2
)

# 调用 API 获取模型回复
response = client.chat.completions.create(
    model="deepseek-chat",  # 3
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}]
    )

# 打印模型回复内容
print(response.choices[0].message.content)

您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-V3。如您有任何任何问题，我会尽我所能为您提供帮助。


# 多轮对话演示

扩展上面的代码，支持多轮对话。这意味着模型可以记住上下文，从而生成更连贯的回答。

## 阿里 API

In [7]:
from openai import OpenAI
import os

# 初始化OpenAI客户端
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'), 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 初始化对话历史
messages = [{"role": "system", "content": "You are a helpful assistant."}]

# 进行多轮对话，当前为3轮
for i in range(3):
    # 获取用户输入
    user_input = input("请输入：")
    
    # 添加用户消息到对话历史
    messages.append({"role": "user", "content": user_input})
    
    # 调用API获取模型回复
    response = client.chat.completions.create(
        model="qwen-turbo",
        messages=messages
    )
    
    # 提取模型回复内容
    assistant_output = response.choices[0].message.content
    
    # 将模型回复添加到对话历史
    messages.append({"role": "assistant", "content": assistant_output})
    
    print(f'用户输入：{user_input}')
    print(f'模型输出：{assistant_output}\n')

请输入： hello


用户输入：hello
模型输出：Hello! How can I assist you today?



请输入： 1+2=


用户输入：1+2=
模型输出：1 + 2 equals 3. Is there anything else you'd like to know or calculate?



请输入： 我们刚刚说了什么


用户输入：我们刚刚说了什么
模型输出：我们刚刚进行了一个简单的数学计算，1 + 2 等于 3。还有什么其他问题或内容需要讨论吗？



## 智谱 API

In [8]:
from openai import OpenAI
import os

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key=os.getenv('ZHIPUAI_API_KEY'),
    base_url="https://open.bigmodel.cn/api/paas/v4",
)

# 初始化对话历史
messages = [{"role": "system", "content": "You are a helpful assistant."}]

# 进行多轮对话，当前为3轮
for i in range(3):
    # 获取用户输入
    user_input = input("请输入：")
    
    # 添加用户消息到对话历史
    messages.append({"role": "user", "content": user_input})
    
    # 调用API获取模型回复
    response = client.chat.completions.create(
        model="glm-4-plus",
        messages=messages
    )
    
    # 提取模型回复内容
    assistant_output = response.choices[0].message.content
    
    # 将模型回复添加到对话历史
    messages.append({"role": "assistant", "content": assistant_output})
    
    print(f'用户输入：{user_input}')
    print(f'模型输出：{assistant_output}\n')

请输入： hello


用户输入：hello
模型输出：Hello! How can I assist you today? If you have any questions or need help with something, feel free to let me know!



请输入： 1+2=


用户输入：1+2=
模型输出：1 + 2 = 3

If you have any more questions or need further assistance, feel free to ask!



请输入： 我们刚刚说了什么


用户输入：我们刚刚说了什么
模型输出：我们刚刚进行了以下对话：

1. 你说：“hello”
2. 我回应：“Hello! How can I assist you today? If you have any questions or need help with something, feel free to let me know!”
3. 你问：“1+2=”
4. 我回答：“1 + 2 = 3”

如果你还有其他问题或需要进一步的帮助，请随时告诉我！



## DeepSeek API

In [9]:
from openai import OpenAI
import os

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key=os.getenv('DEEPSEEK_API_KEY'),
    base_url="https://api.deepseek.com",
)

# 初始化对话历史
messages = [{"role": "system", "content": "You are a helpful assistant."}]

# 进行多轮对话，当前为3轮
for i in range(3):
    # 获取用户输入
    user_input = input("请输入：")
    
    # 添加用户消息到对话历史
    messages.append({"role": "user", "content": user_input})
    
    # 调用API获取模型回复
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    
    # 提取模型回复内容
    assistant_output = response.choices[0].message.content
    
    # 将模型回复添加到对话历史
    messages.append({"role": "assistant", "content": assistant_output})
    
    print(f'用户输入：{user_input}')
    print(f'模型输出：{assistant_output}\n')

请输入： hello


用户输入：hello
模型输出：Hello! How can I assist you today? 😊



请输入： 1+2=


用户输入：1+2=
模型输出：1 + 2 equals **3**. Let me know if you have more questions! 😊



请输入： 我们刚刚说了什么


用户输入：我们刚刚说了什么
模型输出：我们刚刚讨论了简单的加法问题：**1 + 2 = 3**。如果你有其他问题或需要帮助，随时告诉我！ 😊



# 流式输出演示

流式输出允许我们实时查看模型生成的回答，而不是等待最终结果。

## 阿里 API

In [10]:
from openai import OpenAI
import os

# 初始化OpenAI客户端
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 开启流式输出
response = client.chat.completions.create(
    model="qwen-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "你是谁？"}
    ],
    stream=True,
)

# 实时打印模型回复的增量内容
for chunk in response:
    # 判断回复内容是否非空
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end='') 

我是来自阿里云的大规模语言模型，我叫通义千问。

## 智谱 API

In [11]:
from openai import OpenAI
import os

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key=os.getenv('ZHIPUAI_API_KEY'),
    base_url="https://open.bigmodel.cn/api/paas/v4",
)

# 开启流式输出
response = client.chat.completions.create(
    model="glm-4-plus",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "你是谁？"}
    ],
    stream=True,
)

# 实时打印模型回复的增量内容
for chunk in response:
    # 判断回复内容是否非空
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end='') 

我是人工智能助手智谱清言（ChatGLM），是基于智谱 AI 公司于 2024 年训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。

## DeepSeek API

In [12]:
from openai import OpenAI
import os

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key=os.getenv('DEEPSEEK_API_KEY'),
    base_url="https://api.deepseek.com",
)

# 开启流式输出
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "你是谁？"}
    ],
    stream=True,
)

# 实时打印模型回复的增量内容
for chunk in response:
    # 判断回复内容是否非空
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end='') 

您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-V3。如您有任何任何问题，我会尽我所能为您提供帮助。

# 总结与扩展学习

恭喜你完成了 API 的使用！在这个 Notebook 中，我们学习了如何与大语言模型API进行交互，设置环境变量和实现单轮和多轮对话以及流式输出。

如果对 AI 和生成式人工智能感兴趣，推荐观看李宏毅老师的课程：
- [生成式人工智能导论-视频](https://www.bilibili.com/video/BV1BJ4m1e7g8)
- [课程主页](https://speech.ee.ntu.edu.tw/~hylee/genai/2024-spring.php)

# 下一步

- 可以遵循导论的顺序进行阅读，如果对 API 的使用感兴趣，推荐跳转阅读 《[DeepSeek 使用手册](../README.md#deepseek-使用手册doing)》。
- 对流式输出感到疑惑的同学阅读：《[DeepSeek API 输出解析 - OpenAI SDK](../Guide/DeepSeek%20API%20输出解析%20-%20OpenAI%20SDK.md)》和 《[流式输出解析](../Guide/DeepSeek%20API%20流式输出解析%20-%20OpenAI%20SDK.md)》。